NOTE:
-----

Please run the below cells first before proceeding- you'll need them soon!

In [2]:
%load_ext sql
%sql sqlite://

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


'Connected: @None'

In [3]:
%%sql
pragma foreign_keys = ON; -- WARNING: by default off in sqlite
drop table if exists product; -- This needs to be dropped if exists, see why further down!
drop table if exists company;
create table company (
    cname varchar primary key, -- company name uniquely identifies the company.
    stockprice money, -- stock price is in money 
    country varchar); -- country is just a string
insert into company values ('GizmoWorks', 25.0, 'USA');
insert into company values ('Canon', 65.0, 'Japan');
insert into company values ('Hitachi', 15.0, 'Japan');
create table product(
       pname varchar primary key, -- name of the product
       price money, -- price of the product
       category varchar, -- category
       manufacturer varchar, -- manufacturer
       foreign key (manufacturer) references company(cname));
insert into product values('Gizmo', 19.99, 'Gadgets', 'GizmoWorks');
insert into product values('SingleTouch', 149.99, 'Photography', 'Canon');
insert into product values('PowerGizmo', 29.99, 'Gadgets', 'GizmoWorks');
insert into product values('MultiTouch', 203.99, 'Household', 'Hitachi');

 * sqlite://
Done.
Done.
Done.
Done.
1 rows affected.
1 rows affected.
1 rows affected.
Done.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.


[]

In [4]:
%%sql
DROP TABLE IF EXISTS franchise;
CREATE TABLE franchise (name TEXT, db_type TEXT);
INSERT INTO franchise VALUES ('Bobs Bagels', 'NoSQL');
INSERT INTO franchise VALUES ('eBagel', 'NoSQL');
INSERT INTO franchise VALUES ('BAGEL CORP', 'MySQL');

DROP TABLE IF EXISTS store;
CREATE TABLE store (franchise TEXT, location TEXT);
INSERT INTO store VALUES ('Bobs Bagels', 'NYC');
INSERT INTO store VALUES ('eBagel', 'PA');
INSERT INTO store VALUES ('BAGEL CORP', 'Chicago');
INSERT INTO store VALUES ('BAGEL CORP', 'NYC');
INSERT INTO store VALUES ('BAGEL CORP', 'PA');

DROP TABLE IF EXISTS bagel;
CREATE TABLE bagel (name TEXT, price MONEY, made_by TEXT);
INSERT INTO bagel VALUES ('Plain with shmear', 1.99, 'Bobs Bagels');
INSERT INTO bagel VALUES ('Egg with shmear', 2.39, 'Bobs Bagels');
INSERT INTO bagel VALUES ('eBagel Drinkable Bagel', 27.99, 'eBagel');
INSERT INTO bagel VALUES ('eBagel Expansion Pack', 1.99, 'eBagel');
INSERT INTO bagel VALUES ('Plain with shmear', 0.99, 'BAGEL CORP');
INSERT INTO bagel VALUES ('Organic Flax-seed bagel chips', 0.99, 'BAGEL CORP');

DROP TABLE IF EXISTS purchase;
-- Note that date is an int here just to simplify things
CREATE TABLE purchase (bagel_name TEXT, franchise TEXT, date INT, quantity INT, purchaser_age INT);
INSERT INTO purchase VALUES ('Plain with shmear', 'Bobs Bagels', 1, 12, 28);
INSERT INTO purchase VALUES ('Egg with shmear', 'Bobs Bagels', 2, 6, 47);
INSERT INTO purchase VALUES ('Plain with shmear', 'BAGEL CORP', 2, 12, 24);
INSERT INTO purchase VALUES ('Plain with shmear', 'BAGEL CORP', 3, 1, 17);
INSERT INTO purchase VALUES ('eBagel Expansion Pack', 'eBagel', 1, 137, 5);
INSERT INTO purchase VALUES ('Plain with shmear', 'Bobs Bagels', 4, 24, NULL);

 * sqlite://
Done.
Done.
1 rows affected.
1 rows affected.
1 rows affected.
Done.
Done.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
Done.
Done.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
Done.
Done.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.


[]

Order By Reprise
================
* SQL-89 forbids the following, but we can make sense of it:
> SELECT pname FROM Product ORDER BY Price

Let's try out some other slightly unintuitive ORDER BY queries:

In [5]:
%sql SELECT * FROM Product;

 * sqlite://
Done.


pname,price,category,manufacturer
Gizmo,19.99,Gadgets,GizmoWorks
SingleTouch,149.99,Photography,Canon
PowerGizmo,29.99,Gadgets,GizmoWorks
MultiTouch,203.99,Household,Hitachi


In [6]:
%%sql SELECT pname,price FROM Product
ORDER BY pname

 * sqlite://
Done.


pname,price
Gizmo,19.99
MultiTouch,203.99
PowerGizmo,29.99
SingleTouch,149.99


In [7]:
%%sql SELECT pname FROM Product
ORDER BY Price

 * sqlite://
Done.


pname
Gizmo
PowerGizmo
SingleTouch
MultiTouch


In [8]:
%%sql SELECT distinct pname FROM Product
ORDER BY Price

 * sqlite://
Done.


pname
Gizmo
PowerGizmo
SingleTouch
MultiTouch


* Some engines- like Postgres- would balk at this, but some engines- like SQLite- do process this... how?

Set Operators
=======
We generate three tables below:
* R is {1,2,3,4,5}
* S is {}
* T is {1,4,7,10}


In [9]:
# Create tables & insert some random numbers
# Note: in Postgresql, try the generate_series function...
%sql DROP TABLE IF EXISTS R; DROP TABLE IF EXISTS S; DROP TABLE IF EXISTS T;
%sql CREATE TABLE R (A int); CREATE TABLE S (A int); CREATE TABLE T (A int);
for i in range(1,6):
    %sql INSERT INTO R VALUES (:i)
for i in range(1,11,3):
    %sql INSERT INTO T VALUES (:i)

 * sqlite://
Done.
Done.
Done.
 * sqlite://
Done.
Done.
Done.
 * sqlite://
1 rows affected.
 * sqlite://
1 rows affected.
 * sqlite://
1 rows affected.
 * sqlite://
1 rows affected.
 * sqlite://
1 rows affected.
 * sqlite://
1 rows affected.
 * sqlite://
1 rows affected.
 * sqlite://
1 rows affected.
 * sqlite://
1 rows affected.


Let's try to get $R \cap (S \cup T) = \{1,4\}$

In [10]:
%%sql SELECT DISTINCT R.A FROM R, S, T
WHERE R.A=S.A OR R.A=T.A

 * sqlite://
Done.


A


**Why is the return set empty?**

*Recall the order of operations for this query:*
1. We take the cross-product of `R,S,T`
2. We filter this intermediate table by the conditions of the `WHERE` clause.  

Let's see (1):

In [11]:
%sql SELECT DISTINCT R.A FROM R, S, T;

 * sqlite://
Done.


A


The cross-product query is empty because `S` is empty!

Union
-----

Let's try a different way using a new SQL command, `UNION`:

In [12]:
%%sql
SELECT R.A FROM R, S WHERE R.A=S.A
UNION -- this is an explicit keyword!
SELECT R.A FROM R, T WHERE R.A=T.A

 * sqlite://
Done.


A
1
4


* There are no duplicates (union is a set)
* If we wanted duplicates, we could use UNION ALL
  * R = {1,2,3,4,5}
  * S = {1,2,3,4,5}
  * T = {1,4,7,10}

In [13]:
%sql DROP TABLE IF EXISTS S; CREATE TABLE S (A int);
for i in range(1,6):
    %sql INSERT INTO S VALUES (:i)

 * sqlite://
Done.
Done.
 * sqlite://
1 rows affected.
 * sqlite://
1 rows affected.
 * sqlite://
1 rows affected.
 * sqlite://
1 rows affected.
 * sqlite://
1 rows affected.


In [14]:
%%sql
-- UNION ALL example- notice that 1 and 4 occur twice!
SELECT R.A FROM R, S WHERE R.A=S.A
UNION ALL
SELECT R.A FROM R, T WHERE R.A=T.A

 * sqlite://
Done.


A
1
2
3
4
5
1
4


Other Set Operations: INTERSECT, EXCEPT
-------------------------------------

Two examples below... see more in the next section

In [15]:
%%sql
SELECT R.A FROM R, S, T WHERE R.A = S.A
INTERSECT
SELECT R.A FROM R, S, T WHERE R.A = T.A

 * sqlite://
Done.


A
1
4


In [16]:
%%sql
SELECT R.A FROM R, S, T WHERE R.A = S.A
EXCEPT
SELECT R.A FROM R, S, T WHERE R.A = T.A

 * sqlite://
Done.


A
2
3
5


Running example for rest of this notebook: BAGELS
===========================

The selection of which had nothing to do with how hungry the person writing this section was.

Outline of topics:
* Nested queries: motivation & examples
* Aggregation: Overview, with group-by
* Null Values, Outer vs. Inner Joins

Story:
------
* *eBagel* is a ground-breaking new tech startup in the red-hot NoSQL-based bagel enterprise space
* eBagel has just received $100M in venture capital funding.  However, their sales numbers are dropping off, and you have been brought in to analyze the data and figure out what's going wrong

Let's begin by populating the following tables:
> Franchise(name TEXT, db_type TEXT)

> Store(franchise TEXT, location TEXT)

> Bagel(name TEXT, price MONEY, made_by TEXT)

> Purchase(bagel_name TEXT, franchise TEXT, date INT, quantity INT, purchaser_age INT)

Let's quickly load the data first

Union
-----

First, let's find franchises that have locations in PA **OR** NYC to see who the potential competition is:

In [17]:
%%sql 
SELECT franchise FROM store WHERE location = 'NYC'
UNION
SELECT franchise FROM store WHERE location = 'PA';

 * sqlite://
Done.


franchise
BAGEL CORP
Bobs Bagels
eBagel


Intersect: Subtle problem...
--------------------------

eBagel's CEO is curious to know more about the back-end technologies of Bagel companies that have successfully spread across multiple locations.  Let's try to use an `INTERSECT` operator to find the database types of franchises which have stores in PA **AND** NYC:

In [18]:
%%sql
SELECT f.db_type
FROM franchise f, store s 
WHERE f.name = s.franchise AND s.location = 'NYC'
INTERSECT
SELECT f.db_type
FROM franchise f, store s 
WHERE f.name = s.franchise AND s.location = 'PA'

 * sqlite://
Done.


db_type
MySQL
NoSQL


*What happened here?*

If we look at the data, we should only have gotten "MySQL" as a result:

In [19]:
%%sql 
SELECT f.name, s.location, f.db_type
FROM franchise f, store s 
WHERE f.name = s.franchise;

 * sqlite://
Done.


name,location,db_type
Bobs Bagels,NYC,NoSQL
eBagel,PA,NoSQL
BAGEL CORP,Chicago,MySQL
BAGEL CORP,NYC,MySQL
BAGEL CORP,PA,MySQL


So what happened?  We can see why this occurs by breaking the query operation down:

In [20]:
%%sql SELECT f.db_type 
FROM franchise f, store s 
WHERE f.name = s.franchise AND s.location = 'NYC'

 * sqlite://
Done.


db_type
NoSQL
MySQL


In [21]:
%%sql SELECT f.db_type
FROM franchise f, store s
WHERE f.name = s.franchise AND s.location = 'PA'

 * sqlite://
Done.


db_type
NoSQL
MySQL


Essentially, the problem is that we did the `INTERSECT` operation over the return attributes, not the attributes we actually wanted to use!

Nested queries
========

Instead, let's see one solution to the above problem that uses *nested queries*: i.e. subqueries that return relations which our main query then operates over:

In [22]:
%%sql
SELECT f.db_type
FROM franchise f
WHERE f.name IN (
    SELECT s.franchise FROM store s WHERE s.location = 'NYC')
  AND f.name IN (
    SELECT s.franchise FROM store s WHERE s.location = 'PA');

 * sqlite://
Done.


db_type
MySQL


Another nested queries example: eBagel's CEO wants to know what databases support bagel companies that appeal to to the crucial twenty-something demographic:

In [23]:
%%sql
SELECT f.db_type
FROM franchise f
WHERE f.name IN (
    SELECT b.made_by
    FROM bagel b, purchase p
    WHERE b.name = p.bagel_name 
      AND p.purchaser_age >= 20 AND p.purchaser_age < 30);

 * sqlite://
Done.


db_type
NoSQL
MySQL


Nested queries are powerful and convenient!  Could we accomplish the above without them however?  Would this return the same result?

In [24]:
%%sql
SELECT f.db_type
FROM franchise f, bagel b, purchase p
WHERE f.name = b.made_by 
  AND b.name = p.bagel_name 
  AND p.purchaser_age >= 20 AND p.purchaser_age < 30;

 * sqlite://
Done.


db_type
MySQL
NoSQL
MySQL
NoSQL


**Beware of duplicates!**  We can add a `DISTINCT` however and they will be equivalent- try this for yourself.

We can also use the following over the results of nested subqueries:
* `ALL`
* `ANY`
* `EXISTS`

**Unfortunately, `ALL` and `ANY` are not supported in SQLite (the DBMS we're using here)- see the lecture notes for examples.**  However we can show an example of `EXISTS`.  Suppose we want to answer the following question: does eBagel have *any* products which are cheaper than *any* of its competitors'?

In [25]:
%%sql
SELECT b.name, b.price
FROM bagel b
WHERE b.made_by = 'eBagel'
  AND EXISTS (SELECT name FROM bagel WHERE made_by <> 'eBagel' AND price > b.price);

 * sqlite://
Done.


name,price
eBagel Expansion Pack,1.99


Note that this is a *correlated query* because values from the main outer query are involved in the inner nested query.  Especially take notice of the variable scoping here.  Note also that this can be expressed as a single SFW query- can you figure out how?

Summary so far:
--------------

SQL:
* Provides a high-level declarative language for manipulating data (DML)
* The workhorse is the SFW block
* Set operators are powerful- but have some subtleties to be aware of!
* Powerful, nested queries are also allowed
    * **Note that this becomes especially helpful when using a more full-featured DBMS than SQLite!**

Aggregation
=======

SQL supports several aggregation operations:
* `SUM`
* `COUNT`
* `AVG`
* `MIN`
* `MAX`

*Except `COUNT`*, all aggregation operators apply to a single attribute only

Here are some simple examples:

In [26]:
%sql SELECT AVG(price) FROM bagel WHERE made_by = 'eBagel';

 * sqlite://
Done.


AVG(price)
14.989999999999998


Note that you can define the attribute labels of the output schema, which is convenient for aggregation operations:

In [27]:
%sql SELECT COUNT(*) AS "Number of Stores in PA" FROM store WHERE location = 'PA';

 * sqlite://
Done.


Number of Stores in PA
2


What if we want to know how many total locations there are of relevance in the Bagel 2.0 space?

In [28]:
%sql SELECT COUNT(location) FROM store;

 * sqlite://
Done.


COUNT(location)
5


What we actually wanted:

In [29]:
%sql SELECT COUNT(DISTINCT location) FROM store;

 * sqlite://
Done.


COUNT(DISTINCT location)
3


Next, you've purchased some industry sales data, and want to get a sense of how big the bagel space is exactly.  How can we do this easily and efficiently, *all in SQL*?

In [30]:
%%sql
SELECT SUM(b.price * p.quantity) AS net_sales
FROM bagel b, purchase p
WHERE b.name = p.bagel_name;

 * sqlite://
Done.


net_sales
432.99


That's a huge market opportunity!  Now, eBagel's CEO is quite excited, and wants to see more detail, such as how the distribution across companies breaks down.  Luckily, **grouping and aggregation can be combined!**

In [31]:
%%sql
SELECT b.made_by, SUM(b.price * p.quantity) AS revenue
FROM bagel b, purchase p
WHERE b.made_by = p.franchise AND b.name = p.bagel_name
GROUP BY b.made_by;

 * sqlite://
Done.


made_by,revenue
BAGEL CORP,12.87
Bobs Bagels,85.98
eBagel,272.63


These are exciting results!

Finally, to cap off the analysis, you want to see the distribution of revenue by product.  *However*, in order to paint a more effective high-level picture, you only want to see best-sellers, which you define as products with more that a dozen total sales.

To do this we introduce the `HAVING` clause, which conditions *on aggregates*

In [32]:
%%sql
SELECT b.name, SUM(b.price * p.quantity) AS sales
FROM bagel b, purchase p
WHERE b.name = p.bagel_name AND b.made_by = p.franchise
GROUP BY b.name
HAVING SUM(p.quantity) > 12;

 * sqlite://
Done.


name,sales
Plain with shmear,84.50999999999999
eBagel Expansion Pack,272.63


Let's break down this last query into its actual evaluation steps:

First, the FROM-WHERE segment of the clause is evaluated:

In [33]:
%%sql
SELECT *
FROM bagel b, purchase p
WHERE b.name = p.bagel_name AND b.made_by = p.franchise;

 * sqlite://
Done.


name,price,made_by,bagel_name,franchise,date,quantity,purchaser_age
Plain with shmear,1.99,Bobs Bagels,Plain with shmear,Bobs Bagels,1,12,28
Plain with shmear,1.99,Bobs Bagels,Plain with shmear,Bobs Bagels,4,24,None
Egg with shmear,2.39,Bobs Bagels,Egg with shmear,Bobs Bagels,2,6,47
eBagel Expansion Pack,1.99,eBagel,eBagel Expansion Pack,eBagel,1,137,5
Plain with shmear,0.99,BAGEL CORP,Plain with shmear,BAGEL CORP,2,12,24
Plain with shmear,0.99,BAGEL CORP,Plain with shmear,BAGEL CORP,3,1,17


(*Aside: what would we have gotten if we had left out `AND b.made_by = p.franchise` in the WHERE clause?  Breaking down a complex query into its component steps is the best way to debug things like this!*)

Next, the GROUP-BY is applied:

In [34]:
%%sql
SELECT 
    b.name,
    GROUP_CONCAT(b.price, ',') AS prices,
    GROUP_CONCAT(b.made_by, ',') AS made_bys,
    bagel_name, 
    GROUP_CONCAT(p.franchise, ',') AS franchises,
    GROUP_CONCAT(p.date, ',') AS dates,
    GROUP_CONCAT(p.quantity, ',') AS quantities,
    GROUP_CONCAT(p.purchaser_age, ',') AS purchaser_ages
FROM bagel b, purchase p
WHERE b.name = p.bagel_name AND b.made_by = p.franchise
GROUP BY b.name;

 * sqlite://
Done.


name,prices,made_bys,bagel_name,franchises,dates,quantities,purchaser_ages
Egg with shmear,2.39,Bobs Bagels,Egg with shmear,Bobs Bagels,2,6,47
Plain with shmear,"1.99,1.99,0.99,0.99","Bobs Bagels,Bobs Bagels,BAGEL CORP,BAGEL CORP",Plain with shmear,"Bobs Bagels,Bobs Bagels,BAGEL CORP,BAGEL CORP","1,4,2,3","12,24,12,1","28,24,17"
eBagel Expansion Pack,1.99,eBagel,eBagel Expansion Pack,eBagel,1,137,5


*Note that the `GROUP_CONCAT` functions above are just for nicer display, as otherwise SQLite will choose a single arbitrary relation's values to display for the column in such a query*

Next, the `HAVING` condition is applied:

In [35]:
%%sql
SELECT 
    b.name,
    GROUP_CONCAT(b.price, ',') AS prices,
    GROUP_CONCAT(b.made_by, ',') AS made_bys,
    bagel_name, 
    GROUP_CONCAT(p.franchise, ',') AS franchises,
    GROUP_CONCAT(p.date, ',') AS dates,
    SUM(p.quantity) AS total_quantity,
    GROUP_CONCAT(p.purchaser_age, ',') AS purchaser_ages
FROM bagel b, purchase p
WHERE b.name = p.bagel_name AND b.made_by = p.franchise
GROUP BY b.name
HAVING SUM(p.quantity) > 12;

 * sqlite://
Done.


name,prices,made_bys,bagel_name,franchises,dates,total_quantity,purchaser_ages
Plain with shmear,"1.99,1.99,0.99,0.99","Bobs Bagels,Bobs Bagels,BAGEL CORP,BAGEL CORP",Plain with shmear,"Bobs Bagels,Bobs Bagels,BAGEL CORP,BAGEL CORP","1,4,2,3",49,"28,24,17"
eBagel Expansion Pack,1.99,eBagel,eBagel Expansion Pack,eBagel,1,137,5


And finally, the aggregates and other operations in the select clause (projection) are applied:

In [36]:
%%sql
SELECT b.name, SUM(b.price * p.quantity) AS sales
FROM bagel b, purchase p
WHERE b.name = p.bagel_name AND b.made_by = p.franchise
GROUP BY b.name
HAVING SUM(p.quantity) > 12;

 * sqlite://
Done.


name,sales
Plain with shmear,84.50999999999999
eBagel Expansion Pack,272.63


Quantifiers: existential and universal
------------------------------------

A massive amount of user interviewing has suggested that something called "shmear" (or *schmear*) is of critical importance to market success.  You decide to look for competitors that have some shmear on the menu:

In [37]:
%sql SELECT DISTINCT made_by FROM bagel WHERE name LIKE '%shmear%';

 * sqlite://
Done.


made_by
Bobs Bagels
BAGEL CORP


A condition of this form (*"there exists some r s.t. C(r)"*) is known as an **existential** quantifier.  As is apparent above, these are fairly easy to write in SQL.  A **universal** quantifier on the other hand (of the form *"C(r) for all r"*) is a bit harder but still simple enough.

So, for example, to find competitors with products that *all* have shmear in them:

In [38]:
%%sql
SELECT DISTINCT made_by
FROM bagel
WHERE made_by NOT IN (
    SELECT made_by
    FROM bagel
    WHERE name NOT LIKE '%shmear%');

 * sqlite://
Done.


made_by
Bobs Bagels


NULL values in SQL
-----------------

Let's look at an odd pair of queries:

In [39]:
%sql SELECT * FROM purchase WHERE bagel_name LIKE '%shmear%';

 * sqlite://
Done.


bagel_name,franchise,date,quantity,purchaser_age
Plain with shmear,Bobs Bagels,1,12,28
Egg with shmear,Bobs Bagels,2,6,47
Plain with shmear,BAGEL CORP,2,12,24
Plain with shmear,BAGEL CORP,3,1,17
Plain with shmear,Bobs Bagels,4,24,None


In [40]:
%%sql SELECT * FROM purchase 
WHERE bagel_name LIKE '%shmear%' 
  AND (purchaser_age >= 5 OR purchaser_age < 5);

 * sqlite://
Done.


bagel_name,franchise,date,quantity,purchaser_age
Plain with shmear,Bobs Bagels,1,12,28
Egg with shmear,Bobs Bagels,2,6,47
Plain with shmear,BAGEL CORP,2,12,24
Plain with shmear,BAGEL CORP,3,1,17


We see that `NULL` values are treated specially.  In SQL, there are actually three effective boolean values- `TRUE`, `FALSE`, and `UNKNOWN`.  Any comparison operation between a `NULL` value and a constant will return `UNKNOWN`- and in SQL, relations are only passed through when the condition over them returns `TRUE`.  We can of course handle them specially though:

In [41]:
%%sql SELECT * FROM purchase
WHERE bagel_name LIKE '%shmear%'
  AND (purchaser_age >= 5 OR purchaser_age < 5 
       OR purchaser_age IS NULL);

 * sqlite://
Done.


bagel_name,franchise,date,quantity,purchaser_age
Plain with shmear,Bobs Bagels,1,12,28
Egg with shmear,Bobs Bagels,2,6,47
Plain with shmear,BAGEL CORP,2,12,24
Plain with shmear,BAGEL CORP,3,1,17
Plain with shmear,Bobs Bagels,4,24,None


What happens when there are nulls in a join?

In [42]:
%%sql 
SELECT DISTINCT b.name 
FROM bagel b, purchase p 
WHERE b.name = p.bagel_name AND b.made_by = p.franchise;

 * sqlite://
Done.


name
Plain with shmear
Egg with shmear
eBagel Expansion Pack


We're missing bagels which were never purchased!

Inner and Outer Joins
--------------------

A join query using a `WHERE` clause like the one just showed is actually an `INNER JOIN`, and can also be written as follows:

In [43]:
%%sql 
SELECT DISTINCT b.name 
FROM bagel b
    INNER JOIN purchase p ON b.name = p.bagel_name AND b.made_by = p.franchise;

 * sqlite://
Done.


name
Plain with shmear
Egg with shmear
eBagel Expansion Pack


An `INNER JOIN` on tables `A` and `B` with join condition `C(A,B)` returns only relations `(a,b)` such that `C(a,b) = TRUE`.  If, as in our example above, there is no `b` such that `C(a,b)` is true, then `a` is simply not returned in thr output multiset.

We can use an `OUTER JOIN` instead, however, which comes in three varieties: `LEFT`, `RIGHT`, and `FULL`. 

In our current situation, what we needed was a `LEFT OUTER JOIN`.  A left outer join will also return `(a, NULL)` for left relations `a` such that there is no `b` for which `C(a,b) = TRUE`:

In [44]:
%%sql 
SELECT DISTINCT b.name 
FROM bagel b
    LEFT OUTER JOIN purchase p ON b.name = p.bagel_name AND b.made_by = p.franchise;

 * sqlite://
Done.


name
Plain with shmear
Egg with shmear
eBagel Drinkable Bagel
eBagel Expansion Pack
Organic Flax-seed bagel chips
